In [1]:
import json
import pandas as pd
train = pd.read_json('../input/whats-cooking-kernels-only/train.json')
test = pd.read_json('../input/whats-cooking-kernels-only/test.json')

from sklearn.model_selection import train_test_split
X_train = list(train['ingredients'])
y_train = list(train['cuisine'])
X_test = list(test['ingredients'])

from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))

from nltk.stem.porter import *
stemmer = PorterStemmer()

In [2]:
# deldeldelde=list(train['cuisine'])
# for i in set(deldeldelde):
#     print(deldeldelde.count(i))
#     print(i)

In [3]:
def preprocess_ingredient(ingredients):
    for ingredient in ingredients:
        for i in range(len(ingredient)):
            x = ingredient[i] #'Bertolli® Classico Olive Oil', '(10 oz.) frozen chopped spinach, thawed and squeezed dry' ,'leg of lamb', 'lamb leg'
            x = x.lower() #'bertolli® classico olive oil', '(10 oz.) frozen chopped spinach, thawed and squeezed dry' ,'leg of lamb', 'lamb leg'
            x = re.sub("[^a-z ]", "", x) #'bertolli classico olive oil', ' oz frozen chopped spinach thawed and squeezed dry' ,'leg of lamb', 'lamb leg'
            word_tokens = word_tokenize(x)
            if 'oz' in word_tokens:
                word_tokens.remove('oz')
            filtered_words = [w for w in word_tokens if not w in stop_words] 
            filtered_words.sort() #['bertolli', 'classico', 'oil', 'olive'], ['chopped', 'dry', 'frozen', 'spinach', 'squeezed', 'thawed'], ['lamb', 'leg'], ['lamb', 'leg']
            stemmed_word = [stemmer.stem(word) for word in filtered_words]
            x = ' '.join(stemmed_word) #'bertolli classico oil oliv', 'chop dri frozen spinach squeez thaw' ,'lamb leg', 'lamb leg'
            ingredient[i] = x

In [4]:
preprocess_ingredient(X_train)
preprocess_ingredient(X_test)

In [5]:
vocabs = set()
for ingredient in X_train:
    vocabs.update(ingredient)
vocabs = sorted(vocabs)

pickled_vocabs = set()

import pickle
with open('vocabs.pkl', 'wb') as file:
    pickle.dump(vocabs, file)
    
with open('vocabs.pkl', 'rb') as file:  
    pickled_vocabs = sorted(pickle.load(file))

# print(pickled_vocabs)

In [6]:
def create_bag_of_words(ingredients):
    data_features = list()
    for ingredient in ingredients:
        features = list()
        for item in pickled_vocabs:
            features.append(item in ingredient)
        data_features.append(features)
    return data_features

In [7]:
train_data_features = create_bag_of_words(X_train)

In [8]:
def train_logistic_regression(features, label):
    from sklearn.linear_model import LogisticRegression
    ml_model = LogisticRegression(C = 2,random_state = 0)
    ml_model.fit(features, label)
    return ml_model

In [9]:
ml_model = train_logistic_regression(train_data_features, y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [10]:
# import pickle
# with open('cuisine_prediction_model.pkl', 'wb') as file:
#     pickle.dump(ml_model, file)
    
# with open('cuisine_prediction_model.pkl', 'rb') as file:  
#     pickled_ml_model = pickle.load(file)

import joblib
joblib.dump(ml_model, 'cuisine_prediction_model_joblib.pkl')
pickled_ml_model = joblib.load('cuisine_prediction_model_joblib.pkl') 

test_data_features = create_bag_of_words(X_test)
predicted_y = pickled_ml_model.predict(test_data_features)

test['cuisine'] = predicted_y
test[['id', 'cuisine']].to_csv('submission.csv', index=False)
test[['id', 'cuisine']].head()

,id,cuisine
0,18009,british
1,28583,southern_us
2,41580,italian
3,29752,cajun_creole
4,35687,italian


In [11]:
!pip freeze > '../working/dockerimage_snapshot.txt'